In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../')

from evml.keras.callbacks import get_callbacks, ReportEpoch
from evml.keras.models import CategoricalDNN

In [2]:

from evml.keras.losses import DirichletInformedPriorLoss

In [3]:
#load data
df = pd.read_csv("dirichletIP_test_data.csv")

In [4]:
x = df[["x", "y"]].to_numpy()
y = pd.get_dummies(df.labels).to_numpy() #one hot encoding


In [5]:
prior = [1 - 0.375, 0.375]
y = np.hstack((y, np.ones((x.shape[0], 2)) * prior))

In [6]:
#train test split:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
callbacks = []
callbacks.append(ReportEpoch(2.0))

mlp = CategoricalDNN(loss="dirichletIP", output_activation="relu", annealing_coeff=2.0, callbacks=callbacks,
                    optimizer='sgd', epochs = 100)

In [16]:
mlp.loss

'dirichletIP'

In [17]:
# train the model
history = mlp.fit(X_train, y_train)

In [18]:
mlp.predict(X_test)

array([[0.36469403, 0.3686066 ],
       [1.5616746 , 0.03317738],
       [1.3351871 , 0.        ],
       [0.5578107 , 0.3365858 ],
       [0.8006869 , 0.11168545],
       [0.13421305, 0.43558875],
       [0.5617158 , 0.3359383 ],
       [1.0036106 , 0.2021443 ],
       [0.36231202, 0.25232673],
       [0.5617158 , 0.3359383 ],
       [1.4788455 , 0.        ],
       [0.26093653, 0.28630525],
       [1.1908451 , 0.        ],
       [0.44991285, 0.22343098],
       [0.5617158 , 0.3359383 ],
       [1.1176512 , 0.16761592],
       [0.5617158 , 0.3359383 ],
       [0.18063644, 0.37160048],
       [2.1590388 , 0.        ],
       [2.6486976 , 0.        ],
       [0.5617158 , 0.3359383 ],
       [0.88118243, 0.08688754],
       [2.5003126 , 0.        ],
       [0.911367  , 0.0776678 ],
       [2.3639016 , 0.        ],
       [0.66278166, 0.30533826],
       [0.5617158 , 0.3359383 ],
       [0.5617158 , 0.3359383 ],
       [0.6451218 , 0.16048974],
       [0.29176998, 0.38069817],
       [0.

In [ ]:
plt.scatter(np.max)